<a href="https://colab.research.google.com/github/JoaoVitorSantiagoNogueira/case-brick-seguros/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extração
Aqui nós extraimos os dados da internet dos carros

In [86]:
import requests
import re
import pandas as pd

from threading import Thread

## Extrair todos os cógidos FIPE

Feito manualmente, todos os códigos FIPE estão presentes na pagina:
https://www.tabelafipebrasil.com/fipe/carros
bastou fazer um regex no formato [\d]+-[\d]


arquivo gerado ficou diisponível em "https://github.com/JoaoVitorSantiagoNogueira/case-brick-seguros/blob/main/FIPE-CODES.txt"

In [27]:
def get_fipe_codes():
  codes_string = requests.get("https://raw.githubusercontent.com/JoaoVitorSantiagoNogueira/case-brick-seguros/main/FIPE-CODES.txt").text
  codes_list = re.split("\r\n",codes_string)
  return codes_list

In [107]:
len(get_fipe_codes())

6893

In [166]:
get_fipe_codes()[922:933]

['004166-1',
 '004167-0',
 '004168-8',
 '004169-6',
 '004171-8',
 '004172-6',
 '004173-4',
 '004174-2',
 '004175-0',
 '004176-9',
 '004177-7']

## Pegar Informações de Veiculo

In [29]:
# https://brasilapi.com.br/api/fipe/preco/v1/038003-2  Últimos números são o código fipe

def get_json_from_fipe(Codigo_FIPE, query = ""):

  if query != "":
    query = "?tabela_referencia="+query

  response = requests.get("https://brasilapi.com.br/api/fipe/preco/v1/" + Codigo_FIPE + query)
  return response.json()


In [26]:
#sample
type(get_json_from_fipe("038003-2", "9"))

list

#Cria a tabela inteira de um veiculo
Navegando preenchendo mês-a-mês, no formato mm-aa

## Cria o data frame base

In [ ]:
df = pd.json_normalize(get_json_from_fipe("038003-2", "10"))
df

In [163]:
def generate_full_fipe_entry(codigo_fipe, resultados, numero_tread):
    new_line = get_json_from_fipe(codigo_fipe)
    #completar com outros anos de dados da tabela fipe
    print("tread " + str(codigo_fipe) + " has started")
    i=0
    entrada_atual = new_line[0]["mesReferencia"]
    ultima_entrada = None
    while (ultima_entrada != entrada_atual):
      i += 1
      serie_temporal = get_json_from_fipe(codigo_fipe, str(i))

      if type(serie_temporal) == list :
        ultima_entrada = serie_temporal[0]["mesReferencia"]
        for modelo_atual in new_line:
          for modelo_antigo in serie_temporal:
            if (modelo_atual["anoModelo"] == modelo_antigo["anoModelo"] and
               modelo_atual["siglaCombustivel"] == modelo_antigo["siglaCombustivel"]):
               modelo_atual[modelo_antigo["mesReferencia"]] = modelo_antigo["valor"]
               break

    resultados [numero_tread] = new_line
    print("tread " + str(codigo_fipe) + " is done")
    return

In [164]:
#processo demorado, em torno de 1 minuto por veiculo
def create_whole_dataset(codigos):
  #criar um dataframe e atualizar eles é custoso
  #https://stackoverflow.com/questions/13784192/creating-an-empty-pandas-dataframe-and-then-filling-it
  #fazemos primeiro em dicts e depois criamos o data frame
  data_set = []


  num_treads = 10
  threads = [None] * num_treads
  results = [None] * num_treads

  offset= 0 #usado para poder loopar o numero de treads

  created_thread_count = 0
  for i in range(len(threads)): ## cada linha gera sua pesquisa própria
    if i+offset >= len (codigos):
      break
    created_thread_count += 1
    threads[i] = Thread(target = generate_full_fipe_entry, args = (codigos[i+offset], results, i))
    threads[i].start()


  for i in range(created_thread_count):
    threads[i].join()

  print (results)

  return data_set

In [167]:
result = create_whole_dataset(['004166-1',
 '004167-0',
 '004168-8',
 '004169-6',
 '004171-8',
 '004172-6',
 '004173-4',
 '004174-2',
 '004175-0',
 '004176-9',
 '004177-7'
])
print(result)

tread 004174-2 has started
tread 004172-6 has started
tread 004176-9 has started
tread 004168-8 has started
tread 004166-1 has started
tread 004169-6 has startedtread 004167-0 has started

tread 004173-4 has started
tread 004171-8 has started
tread 004175-0 has started
tread 038002-4 is done
tread 004166-1 is done
tread 004167-0 is done
tread 004176-9 is done
tread 004171-8 is done
tread 004172-6 is done
tread 004169-6 is done
tread 004174-2 is done
tread 004175-0 is done
tread 004168-8 is done
tread 004173-4 is done
[[{'valor': 'R$ 23.407,00', 'marca': 'GM - Chevrolet', 'modelo': 'Sonoma CE 4.3 V6', 'anoModelo': 1995, 'combustivel': 'Gasolina', 'codigoFipe': '004166-1', 'mesReferencia': 'maio de 2024 ', 'tipoVeiculo': 1, 'siglaCombustivel': 'G', 'dataConsulta': 'segunda-feira, 13 de maio de 2024 17:17', 'setembro de 2005 ': 'R$ 17.782,00', 'outubro de 2005 ': 'R$ 17.792,00', 'novembro de 2005 ': 'R$ 17.854,00', 'agosto de 2005 ': 'R$ 17.735,00', 'janeiro de 2006 ': 'R$ 17.946,00', 'fe